In [18]:
# loading in modules
import sqlite3
import sys
# from peewee import *
import pandas as pd 
from sudulunu.helpers import pp, make_num, dumper, rc

from transformers import pipeline

## Zero shot classifer
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")
# https://huggingface.co/facebook/bart-large-mnli

## Text classification/clustering
# https://huggingface.co/docs/transformers/tasks/sequence_classification

import os 
import pathlib
pathos = pathlib.Path(os.path.abspath('zero_shot.ipynb')).parent
os.chdir(pathos)
print(os.getcwd())


/Users/josh_nicholas/Repos/oz-2023-facebook-political-ads-scraper/classify_model


In [19]:

db = 'input/scraperwiki.sqlite'
con = sqlite3.connect(db)

cur = con.cursor()

# table_list = [a for a in cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")]
# print(table_list)
# [('ads_by_id',), ('ads_by_query',)]

# key_list = [x for x in cur.execute('SELECT * FROM ads_by_query')]
entries = [x for x in cur.execute('SELECT * FROM ads_by_id')]

# advertisers = []
# for thingo in entries:
#     advertisers.append(thingo[2])
# print(list(set(advertisers)))

# ['From the Heart', 'Fair Australia', 'The Uluru Statement from the Heart', 'Senator Jacinta Nampijinpa Price', 
# 'Not My Voice', 'Referendum News', 'Linda Burney', 'Not Enough', 'Yes23']

con.close()

In [21]:
records = []

candidate_labels = ['other', 'Voice to Parliament']


counter = 0
for thingo in entries:
    counter += 1
    if counter % 50 ==0:
        
        print(counter)
    old = pd.read_csv('inter/zero_shot.csv')
    donners = old['Ad_id'].unique().tolist()
    ad_id = thingo[0]

    if ad_id in donners:
        continue
    else:

        try:

            advertiser_id = thingo[1]
            name = thingo[2]
            texto = thingo[3]
            classy = classifier(texto, candidate_labels)
            
            # print(ad_id)
            # print(classy['labels'][0])
            # print(texto)
            # print(classy['scores'])
            records.append({"Ad_id": ad_id, "Advertiser": advertiser_id, "Classification": classy['labels'][0], "Text": texto})

            cat = pd.DataFrame.from_records(records)
            new = pd.concat([old, cat])
            new.drop_duplicates(subset=['Ad_id'], inplace=True)

            dumper('inter', 'zero_shot', new)

        except Exception as e:

            print(f"Exception is {e}")
            print(f"Line: {sys.exc_info()[-1].tb_lineno}")
            pp(cat)

pp(cat)


50
100
150
200
250
300
350
400
450
Exception is object of type 'NoneType' has no len()
Line: 25
                Ad_id       Advertiser       Classification  \
0    1083098386186304  102329728050606                other   
1    1356150461975366  102329728050606                other   
2     808801313988568  102329728050606                other   
3     839262740550948  102329728050606                other   
4    1024076745433089  102329728050606  Voice to Parliament   
..                ...              ...                  ...   
493   212097394873529  102329728050606  Voice to Parliament   
494   218700897474857  102329728050606                other   
495   214119751244647  102329728050606                other   
496   244886638001228  102329728050606                other   
497  6419524001411199  102329728050606                other   

                                                  Text  
0    Listening to people leads to better outcomes o...  
1    A ‘Yes’ vote for constitutio

In [22]:
zero = pd.read_csv('inter/zero_shot.csv')

zero['Count'] = 1

grp = zero.groupby(by=['Classification'])['Count'].sum().reset_index()

print(grp)

        Classification  Count
0  Voice to Parliament    432
1                other   1497
